In [2]:
!pip install category_encoders

You should consider upgrading via the 'C:\Users\tjddl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
import lightgbm as lgb
from category_encoders import TargetEncoder
from sklearn.metrics import accuracy_score, roc_curve

# 1. 데이터 로드
train = pd.read_csv('train.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])

# 2. 독립 변수(X)와 종속 변수(y) 분리
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

# 3. 카테고리형 변수 인코딩 (Target Encoding)
categorical_columns = [
    "시술 시기 코드", "시술 당시 나이", "시술 유형", "특정 시술 유형", "배란 자극 여부",
    "배란 유도 유형", "단일 배아 이식 여부", "착상 전 유전 검사 사용 여부", "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인", "부부 주 불임 원인",
    "부부 부 불임 원인", "불명확 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태", "배아 생성 주요 이유", "총 시술 횟수",
    "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수", "총 임신 횟수", "IVF 임신 횟수",
    "DI 임신 횟수", "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수", "난자 출처", "정자 출처",
    "난자 기증자 나이", "정자 기증자 나이", "동결 배아 사용 여부", "신선 배아 사용 여부",
    "기증 배아 사용 여부", "대리모 여부", "PGD 시술 여부", "PGS 시술 여부"
]

# Target Encoding 적용
encoder = TargetEncoder(cols=categorical_columns)
X_encoded = encoder.fit_transform(X, y)
test_encoded = encoder.transform(test)

# 4. 데이터 분할 (훈련/검증 데이터셋)
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# 5. LightGBM 모델 학습
final_model = lgb.LGBMClassifier(n_estimators=1000, learning_rate=0.01, max_depth=10, random_state=42)
final_model.fit(X_train, y_train)

# 6. 검증 데이터에 대해 확률 예측
prob_predictions = final_model.predict_proba(X_val)[:, 1]  # 클래스 1의 확률만 추출

# 확률 예측 결과 출력 (상위 10개)
print("검증 데이터에 대한 예측 확률 (상위 10개):")
print(prob_predictions[:10])

# 7. 최적의 임계값 찾기 (ROC 커브)
fpr, tpr, thresholds = roc_curve(y_val, prob_predictions)
optimal_idx = (tpr - fpr).argmax()
optimal_threshold = thresholds[optimal_idx]

print(f"\n최적 임계값: {optimal_threshold}")

# 8. 최적 임계값에 따라 최종 예측 수행
final_predictions = (prob_predictions >= optimal_threshold).astype(int)

# 9. 검증 데이터 정확도 출력
accuracy = accuracy_score(y_val, final_predictions)
print(f"\n최종 검증 데이터 정확도: {accuracy:.4f}")

# 10. 테스트 데이터 예측 확률 및 최종 예측
test_prob_predictions = final_model.predict_proba(test_encoded)[:, 1]  # 테스트 데이터의 클래스 1 확률
test_final_predictions = (test_prob_predictions >= optimal_threshold).astype(int)

# 11. 예측 결과 저장
submission = pd.DataFrame({'ID': pd.read_csv('test.csv')['ID'], '임신 성공 확률': test_prob_predictions, '최종 예측': test_final_predictions})
submission.to_csv('submission.csv', index=False)

print("\n최종 결과가 'submission.csv'에 저장되었습니다.")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 53102, number of negative: 151978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015510 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 779
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051521
[LightGBM] [Info] Start training from score -1.051521
검증 데이터에 대한 예측 확률 (상위 10개):
[0.00091267 0.20905305 0.06350042 0.32096162 0.39091707 0.17815922
 0.34447143 0.25397721 0.41257444 0.32070472]

최적 임계값: 0.257864332102723

최종 검증 데이터 정확도: 0.6161

최종 결과가 'submission.csv'에 저장되었습니다.
